# R-NaD Pokemon TCG Pocket Training on Colab

This notebook sets up the environment and runs the training script for R-NaD on Google Colab.

In [ ]:
# @title Clone Repository
import os

os.chdir("/content")

REPO_URL = "https://github.com/kitamura-tetsuo/R-NaD-PokemonTCGP.git"
REPO_NAME = "R-NaD-PokemonTCGP"

if not os.path.exists(REPO_NAME):
    print(f"Cloning {REPO_NAME}...")
    !git clone --recursive {REPO_URL}
else:
    print(f"{REPO_NAME} already exists. Pulling latest changes...")
    os.chdir(REPO_NAME)
    !git pull
    !git submodule update --init --recursive
    os.chdir("..")

os.chdir(REPO_NAME)
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# @title Setup Environment
import os
import subprocess

# 1. Install Rust (required for deckgym-core)
print("Installing Rust...")
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
os.environ["PATH"] += ":/root/.cargo/bin"

# 2. Install uv
print("Installing uv...")
!curl -LsSf https://astral.sh/uv/install.sh | sh

# 3. Check TPU/GPU and install JAX
try:
    import jax
except ImportError:
    pass

if 'COLAB_TPU_ADDR' in os.environ:
    print("TPU detected. Installing JAX for TPU...")
    !uv pip install --system "jax[tpu]" -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
else:
    print("TPU not detected. Installing JAX for CUDA...")
    !nvidia-smi
    !uv pip install --system "jax[cuda12]"

# 4. Install other dependencies
print("Installing other Python dependencies...")
!uv pip install --system dm-haiku optax open_spiel mlflow orbax-checkpoint maturin

# 5. Build deckgym-core
if os.path.exists("deckgym-core"):
    print("Building deckgym-core...")
    # Using pip install directly since maturin develop requires a venv
    !pip install -v ./deckgym-core
    
    # Copy deckgym_openspiel to current directory to ensure it is importable
    if os.path.exists("deckgym-core/python/deckgym_openspiel"):
        print("Copying deckgym_openspiel to current directory...")
        !cp -r deckgym-core/python/deckgym_openspiel .
else:
    print("Warning: deckgym-core not found. Make sure you have cloned the repository properly.")

In [ ]:
# @title Run Training
import sys
import os

# Add paths to PYTHONPATH
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), "deckgym-core/python"))
os.environ['PYTHONPATH'] = f"{os.getcwd()}:{os.path.join(os.getcwd(), 'deckgym-core/python')}"

# Verify imports
try:
    import deckgym
    import deckgym_openspiel
    print("deckgym and deckgym_openspiel imported successfully.")
except ImportError as e:
    print(f"Import failed: {e}")

# Run script
# You can adjust arguments as needed
!python train.py --max_steps 1000 --save_interval 500